In [1]:
import tensorflow as tf
import numpy as np

from models.Unet3D import unet_3d
import losses

In [2]:
CLASSES = 8
CHANNELS = 3
SIZE = 96

model = unet_3d(class_num = CLASSES, size = SIZE)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 96, 96, 96, 3)]      0         []                            
                                                                                                  
 conv3d (Conv3D)             (None, 96, 96, 96, 32)       2624      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 96, 96, 96, 32)       128       ['conv3d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 leaky_re_lu (LeakyReLU)     multiple                     0         ['batch_normalization[0][0

 batch_normalization_9 (Bat  (None, 6, 6, 6, 512)         2048      ['conv3d_9[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 conv3d_transpose (Conv3DTr  (None, 12, 12, 12, 512)      2097664   ['leaky_re_lu[9][0]']         
 anspose)                                                                                         
                                                                                                  
 concatenate (Concatenate)   (None, 12, 12, 12, 768)      0         ['conv3d_transpose[0][0]',    
                                                                     'leaky_re_lu[7][0]']         
                                                                                                  
 conv3d_10 (Conv3D)          (None, 12, 12, 12, 256)      5308672   ['concatenate[0][0]']         
          

Create some dataset for testing

In [3]:
TRAIN_LENGTH = 10
TEST_LENGTH = 4
BATCH_SIZE = 2

MATRIX = np.random.choice(np.arange(10, dtype=np.float16), size=(CHANNELS, CLASSES), replace=True)

def normalize(data):
    return (data / 127.5) - 1.0

def get_dataset(data_size):
    features = []
    labels = []
    for i in range(data_size):
        data = np.random.choice(np.arange(256, dtype=np.float16), size=(SIZE, SIZE, SIZE, CHANNELS), replace=True)
        features.append(normalize(data))
    labels = np.dot(features, MATRIX)
    return tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(features, dtype=tf.float16), tf.convert_to_tensor(labels, dtype=tf.float16)))

train_dataset = get_dataset(TRAIN_LENGTH)
test_dataset = get_dataset(TEST_LENGTH)

train_batches = train_dataset.batch(BATCH_SIZE)
test_batches = test_dataset.batch(BATCH_SIZE)

In [ ]:
model = unet_3d(class_num = CLASSES, size = SIZE)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=losses.dice_loss,
              metrics=['accuracy'])

model_history = model.fit(train_batches,
                          epochs=3,
                          steps_per_epoch=TRAIN_LENGTH // BATCH_SIZE,
                          validation_steps=TEST_LENGTH // BATCH_SIZE,
                          validation_data=test_batches)

Epoch 1/3


2025-01-12 15:57:59.037372: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


5/5 [==============================] - 965s 194s/step - loss: 0.8957 - accuracy: 0.1324 - val_loss: 0.9936 - val_accuracy: 0.1068
Epoch 2/3
2/5 [===========>..................] - ETA: 5:00 - loss: 0.8484 - accuracy: 0.1356 

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

print(loss)
print(val_loss)

In [ ]:
import matplotlib.pyplot as plt

loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.figure()
plt.plot(model_history.epoch, loss, 'r', label='Training loss')
plt.plot(model_history.epoch, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 2])
plt.legend()
plt.show()

In [5]:
print(tf.__version__)

2.13.0


In [4]:
# check losses are equal to tf implementation
for j, d in enumerate(test_dataset):
    img = tf.expand_dims(d[0], axis=0)
    label = tf.expand_dims(d[1], axis=0)

    prediction = model.predict(img)

    print(f'dice loss: {losses.dice_loss(label, prediction, 0)}')
    print(f'tf dice loss: {tf.keras.losses.Dice()(label, prediction)}')

    print(f'cross entropy: {losses.cross_entropy_tf(label, prediction)}')
    print(f'tf cross entropy: {tf.keras.losses.CategoricalCrossentropy(from_logits=False)(label, prediction)}')
    break

1/1 [==============================] - 3s 3s/step
dice loss: 0.9977645874023438


AttributeError: module 'keras.api._v2.keras.losses' has no attribute 'Dice'